# `duckreg`: Out-of-memory regressions with `duckdb`

Remember to carefully examine the underlying queries that generate the data; the package merely passes it into `np.linalg.lstsq` to solve the least squares problem.

In [1]:
import numpy as np
import pandas as pd
from duckreg.estimators import DuckRegression
import duckdb
import pyfixest as pf

## Regression with discrete covariates

The largest gains from compression arise when the right hand side variables are discrete, which is also the setting where linearity is an innocuous assumption (especially when the RHS is saturated).

In [2]:
# Generate sample data
def generate_sample_data(N=10_000_000, seed=12345):
    rng = np.random.default_rng(seed)
    D = rng.choice([0, 1], size=(N, 1))
    X = rng.choice(range(20), (N, 2), True)
    Y = D + X @ np.array([1, 2]).reshape(2, 1) + rng.normal(size=(N, 1))
    Y2 = -1 * D + X @ np.array([1, 2]).reshape(2, 1) + rng.normal(size=(N, 1))
    df = pd.DataFrame(
        np.concatenate([Y, Y2, D, X], axis=1), columns=["Y", "Y2", "D", "f1", "f2"]
    ).assign(rowid=range(N))
    return df


# Function to create and populate DuckDB database
def create_duckdb_database(df, db_name="large_dataset.db", table="data"):
    conn = duckdb.connect(db_name)
    conn.execute(f"DROP TABLE IF EXISTS {table}")
    conn.execute(f"CREATE TABLE {table} AS SELECT * FROM df")
    conn.close()
    print(f"Data loaded into DuckDB database: {db_name}")

In [3]:
# Generate and save data
df = generate_sample_data()
db_name = "large_dataset.db"
create_duckdb_database(df, db_name)

Data loaded into DuckDB database: large_dataset.db


Examine the dataset. This dataset could be unreasonably large, which prevents the use of conventional in-memory regression techniques.

In [4]:
db_name = "large_dataset.db"
conn = duckdb.connect(db_name)
query = "SELECT * FROM data limit 5"
conn.execute(query).fetchdf()

,Y,Y2,D,f1,f2,rowid
0,27.226866,24.444717,1.0,16.0,5.0,0
1,35.088713,35.392007,0.0,1.0,17.0,1
2,22.472292,21.185366,1.0,6.0,8.0,2
3,39.842856,38.012720,0.0,9.0,15.0,3
4,23.136978,22.168634,0.0,10.0,6.0,4


The core idea of compressed representations is to retain the minimal set of sufficient statistics to compute the point estimate and variance-covariance matrix. We first demo the compression query and the resultant dataset manually.

In [5]:
db_name = "large_dataset.db"
conn = duckdb.connect(db_name)
q = """
        SELECT D, f1, f2, COUNT(*) as count,
        SUM(Y) as sum_Y,
        SUM(POW(Y, 2)) as sum_Y_sq,
        FROM data
        GROUP BY D, f1, f2
"""
compressed_df = conn.execute(q).fetchdf()
conn.close()

compressed_df.eval("mean_Y = sum_Y / count", inplace=True)
print(compressed_df.shape)
compressed_df.head()

(800, 7)


,D,f1,f2,count,sum_Y,sum_Y_sq,mean_Y
0,1.0,6.0,13.0,12527,413485.030949,1.366067e+07,33.007506
1,0.0,1.0,15.0,12289,380839.544911,1.181442e+07,30.990280
2,1.0,1.0,14.0,12518,375456.350549,1.127372e+07,29.993318
3,0.0,3.0,7.0,12642,214930.812301,3.666901e+06,17.001330
4,1.0,14.0,7.0,12615,365865.474043,1.062339e+07,29.002416


We've reduced a 1 million observation dataset to 800 observations. This will speed up estimation dramatically.

### Regression

$$
Y_i + X_i \beta + \alpha_i + \epsilon_i
$$


#### analytic HC1 standard errors

The easiest case is linear regression with robust SEs, which can be computed directly from compressed data.


In [6]:
%%time
m = DuckRegression(
    db_name="large_dataset.db",
    table_name="data",
    formula="Y ~ D + f1 + f2",
    cluster_col="",
    n_bootstraps=0,
    seed=42,
)
m.fit()
m.fit_vcov()
results = m.summary()
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)
restab

CPU times: user 1.79 s, sys: 72.9 ms, total: 1.86 s
Wall time: 372 ms


,point_estimate,standard_error
0,-0.000274,0.000862
1,0.999347,0.000632
2,1.000035,0.000055
3,2.000067,0.000055


Compare this to the fully disaggregated data implementation (with fixed effects)

In [7]:
%%time
m_pf = pf.feols("Y ~ D | f1 + f2", df, vcov="hetero")
m_pf.tidy()

CPU times: user 9.07 s, sys: 553 ms, total: 9.63 s
Wall time: 6.18 s


,Estimate,Std. Error,t value,Pr(>|t|),2.5%,97.5%
Coefficient,,,,,,
D,0.999347,0.000632,1580.194126,0.0,0.998107,1.000586


Pyfixest with full data takes ~ 40x longer to compute. 

In [8]:
%%time
import statsmodels.formula.api as smf

m_smf = smf.ols("Y ~ D + C(f1) + C(f2)", df).fit(cov_type="HC1")
m_smf.params.loc["D"], m_smf.bse.loc["D"]

CPU times: user 4min 58s, sys: 39.8 s, total: 5min 38s
Wall time: 2min 23s


(np.float64(0.999346888418407), np.float64(0.0006324203286037654))

The full data run in statsmodels takes around 600x longer than the compressed representation in `DuckRegression`. 

In [ ]:
pd.DataFrame(
    np.c_[
        np.r_[results["point_estimate"][1], results["standard_error"][1]],
        m_pf.tidy().iloc[0][["Estimate", "Std. Error"]].values,
        np.r_[m_smf.params.loc["D"], m_smf.bse.loc["D"]],
    ],
    columns=["duckreg", "pyfixest", "statsmodels"],
    index=["estimate", "std.error"],
)

,duckreg,pyfixest,statsmodels
estimate,0.999347,0.999347,0.999347
std.error,0.000632,0.000632,0.000632


#### cluster-robust standard errors

for clustered data, we will use the cluster bootstrap.

In [8]:
m = DuckRegression(
    db_name="large_dataset.db",
    table_name="data",
    formula="Y ~ D + f1 + f2",
    cluster_col="f1",
    n_bootstraps=100,
    seed=42,
)
m.fit()
results = m.summary()

restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)
restab

100%|██████████| 100/100 [00:33<00:00,  2.95it/s]


,point_estimate,standard_error
0,-0.000274,0.000569
1,0.999347,0.000736
2,1.000035,0.000039
3,2.000067,0.000037


The underlying estimator is powered by the following queries:

In [9]:
print(m.agg_query)
print(m.bootstrap_query)


        SELECT D, f1, f2, COUNT(*) as count, SUM(Y) as sum_Y, SUM(POW(Y, 2)) as sum_Y_sq
        FROM data
        GROUP BY D, f1, f2
        

            SELECT D, f1, f2, COUNT(*) as count, SUM(Y) as sum_Y
            FROM data
            WHERE f1 IN (SELECT unnest((?)))
            GROUP BY D, f1, f2
            


### multiple outcomes

We can run regressions on multiple outcome variables painlessly since we only need to keep track of summary stats; just include all dependent variables on the RHS of the formula. The output of `DuckRegression.fit()` concatenates the results of all regressions into a single table, which can then be sliced to extract the relevant coefficients and SEs.

In [10]:
m2 = DuckRegression(
    db_name="large_dataset.db",
    table_name="data",
    formula="Y + Y2 ~ D + f1 + f2",
    cluster_col="f1",
    n_bootstraps=100,
    seed=232,
)

m2.fit()
print(results := m2.summary())

100%|██████████| 100/100 [00:38<00:00,  2.59it/s]

{'point_estimate': array([-2.73682294e-04, -4.39868205e-04,  9.99347122e-01, -9.99443294e-01,
        1.00003536e+00,  1.00006450e+00,  2.00006685e+00,  1.99995671e+00]), 'standard_error': array([6.70325435e-04, 4.15857348e-04, 7.26599979e-04, 4.45407815e-04,
       4.67005434e-05, 4.95007891e-05, 3.96508809e-05, 3.66987192e-05])}


The compressed data contains summary stats on all outcomes

In [11]:
m2.df_compressed

,D,f1,f2,count,sum_Y,sum_Y2,sum_Y_sq,sum_Y2_sq,mean_Y,mean_Y2
0,1.0,16.0,1.0,12395,235463.726925,210700.599740,4.485553e+06,3.594147e+06,18.996670,16.998838
1,0.0,8.0,7.0,12351,271683.595444,271604.033443,5.988431e+06,5.984957e+06,21.996891,21.990449
2,1.0,12.0,14.0,12579,515509.114135,490622.027746,2.113883e+07,1.914848e+07,40.981725,39.003262
3,0.0,0.0,3.0,12484,75037.646656,74873.009753,4.634795e+05,4.614652e+05,6.010705,5.997518
4,1.0,17.0,18.0,12159,656632.641213,632393.029916,3.547280e+07,3.290315e+07,54.003836,52.010283
...,...,...,...,...,...,...,...,...,...,...
795,1.0,8.0,5.0,12495,237397.967083,212456.025869,4.522934e+06,3.625122e+06,18.999437,17.003283
796,0.0,13.0,19.0,12424,633597.411975,633524.976810,3.232453e+07,3.231684e+07,50.997860,50.992030
797,0.0,14.0,14.0,12648,531095.342089,531232.048632,2.231381e+07,2.232497e+07,41.990460,42.001269
798,1.0,14.0,13.0,12622,517589.531971,492104.093359,2.123756e+07,1.919859e+07,41.006935,38.987806


Output needs some post-processing.

In [12]:
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
    index=[f"{x}_{y}" for x in ["Intercept", "D", "f1", "f2"] for y in ["Y", "Y2"]],
)
restab

,point_estimate,standard_error
Intercept_Y,-0.000274,0.000670
Intercept_Y2,-0.000440,0.000416
D_Y,0.999347,0.000727
D_Y2,-0.999443,0.000445
f1_Y,1.000035,0.000047
f1_Y2,1.000064,0.000050
f2_Y,2.000067,0.000040
f2_Y2,1.999957,0.000037


## Panel Data

We have specialised estimators for panel data with N >> T, where the conventional compressed approach is inefficient.

In [13]:
def sim_panel(
    # Parameters
    N=1_000_000,
    T=35,  # Number of units and time periods
    T0=5,  # Treatment starts at T0
    tau=0.005,
    sigma_list=[5, 2, 0.01, 2],
    seed=42,
):
    np.random.seed(seed)
    sigma_unit, sigma_time, sigma_tt, sigma_e = sigma_list
    # Generate data
    unit_ids = np.repeat(np.arange(N), T)
    time_ids = np.tile(np.arange(T), N)

    # Generate unit-specific intercepts and time trends
    unit_fe = np.random.normal(0, sigma_unit, N)
    time_fe = np.random.normal(
        0, sigma_time, T
    )  # Common shocks for all units at each time period
    unit_tt = np.random.normal(0, sigma_tt, N)

    # Generate treatment indicator (randomly assigned)
    W = np.random.binomial(1, 0.5, N)
    W = np.repeat(W, T)
    W = W * (time_ids >= T0)

    rho = 0.7  # Autoregressive parameter for residuals
    # Generate serially correlated residuals for each unit (optimized version)
    residuals = np.zeros((N, T))
    residuals[:, 0] = np.random.normal(0, sigma_e, N)
    epsilon = np.random.normal(0, 1, (N, T - 1))
    factor = 0.5 * np.sqrt(1 - rho**2)
    for t in range(1, T):
        residuals[:, t] = rho * residuals[:, t - 1] + factor * epsilon[:, t - 1]
    # iid
    # residuals = np.random.normal(0, 0.5, N*T)

    # Generate outcome
    Y = (
        np.repeat(unit_fe, T)
        + np.repeat(unit_tt, T) * time_ids
        + tau * W  # Treatment effect is 1
        + np.tile(time_fe, N)  # time FE
        + residuals.flatten()
    )  # Individual noise

    # Create DataFrame
    df = pd.DataFrame({"unit": unit_ids, "time": time_ids, "Y": Y, "W": W})

    return df


df = sim_panel(tau=1)

In [14]:
db_name, table_name = "panel_data.db", "panel_data"
# write to database
conn = duckdb.connect(db_name)
conn.execute(f"DROP TABLE IF EXISTS {table_name}")
conn.execute(f"CREATE TABLE {table_name} AS SELECT * from df")
conn.close()

Peek at the data

In [15]:
conn = duckdb.connect(db_name)
print(conn.execute("SELECT * FROM panel_data LIMIT 5").fetchdf())
conn.close()

   unit  time         Y  W
0     0     0  3.096240  0
1     0     1  2.656935  0
2     0     2  5.084101  0
3     0     3  3.274999  0
4     0     4  4.638763  0


In [16]:
from duckreg.estimators import DuckMundlak, DuckDoubleDemeaning

### Mundlak


One-way mundlak 

$$
Y_{it} = \alpha + \beta X_{it} + \gamma \bar{X}_{i} + \epsilon_{it}
$$

Two-way mundlak

$$
Y_{it} = \alpha + \beta X_{it} + \gamma \bar{X}_{i, \cdot} + \delta \bar{X}_{\cdot, t} + \epsilon_{it}
$$

both of which can be compressed easily with `duckdb`.

These representations are much more efficient than the above general procedure because the unit and time fixed effects are typically very high dimensional, but covariate averages are not. Also see [Arkhangelsky and Imbens](https://arxiv.org/abs/1807.02099) on this.

In [17]:
mundlak = DuckMundlak(
    db_name="panel_data.db",
    table_name="panel_data",
    outcome_var="Y",
    covariates=["W"],
    unit_col="unit",
    time_col="time",
    cluster_col="unit",
    n_bootstraps=50,
    seed=929,
)
mundlak.fit()

mundlak_results = mundlak.summary()

restab = pd.DataFrame(
    np.c_[mundlak_results["point_estimate"], mundlak_results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)
restab

100%|██████████| 50/50 [01:35<00:00,  1.90s/it]


,point_estimate,standard_error
0,0.896134,0.005633
1,1.003877,0.001765
2,0.009106,0.009642
3,-2.413955,0.002209


Powered by the following sequence of queries

In [18]:
print(mundlak.unit_avg_query)
print(mundlak.time_avg_query)
print(mundlak.design_matrix_query)
print(mundlak.compress_query)
print(mundlak.bootstrap_query)


        CREATE TEMP TABLE unit_avgs AS
        SELECT unit,
               AVG(W) AS avg_W_unit
        FROM panel_data
        GROUP BY unit
        

            CREATE TEMP TABLE time_avgs AS
            SELECT time,
                   AVG(W) AS avg_W_time
            FROM panel_data
            GROUP BY time
            

        CREATE TEMP TABLE design_matrix AS
        SELECT
            t.unit,
            t.time,
            t.Y,
            t.W,
            u.avg_W_unit
            , tm.avg_W_time
        FROM panel_data t
        JOIN unit_avgs u ON t.unit = u.unit
        JOIN time_avgs tm ON t.time = tm.time
        

        SELECT
            W, avg_W_unit, avg_W_time,
            COUNT(*) as count,
            SUM(Y) as sum_Y
        FROM design_matrix
        GROUP BY W, avg_W_unit, avg_W_time
        

            SELECT
                W,
                avg_W_unit
                , avg_W_time,
                COUNT(*) as count,
                SUM(Y) as sum_Y
     

### Double Demeaning

$$
Y_{it} = \alpha + \ddot{X}_{it} \beta + \epsilon_{it}
$$

where $\ddot{X}_{it} = X_{it} - \bar{X}_{i, \cdot} - \bar{X}_{\cdot, t} + \bar{X}$

In [19]:
double_demean = DuckDoubleDemeaning(
    db_name="panel_data.db",
    table_name="panel_data",
    outcome_var="Y",
    treatment_var="W",
    unit_col="unit",
    time_col="time",
    cluster_col="unit",
    n_bootstraps=100,
    seed=828,
)

double_demean.fit()

dd_results = double_demean.summary()

restab = pd.DataFrame(
    np.c_[dd_results["point_estimate"], dd_results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)
restab

100%|██████████| 100/100 [02:44<00:00,  1.64s/it]


,point_estimate,standard_error
0,0.295524,0.003857
1,1.003877,0.002097


In [20]:
print(double_demean.overall_mean_query)
print(double_demean.unit_mean_query)
print(double_demean.time_mean_query)
print(double_demean.double_demean_query)
print(double_demean.compress_query)
print(double_demean.bootstrap_query)


        CREATE TEMP TABLE overall_mean AS
        SELECT AVG(W) AS mean_W
        FROM panel_data
        

        CREATE TEMP TABLE unit_means AS
        SELECT unit, AVG(W) AS mean_W_unit
        FROM panel_data
        GROUP BY unit
        

        CREATE TEMP TABLE time_means AS
        SELECT time, AVG(W) AS mean_W_time
        FROM panel_data
        GROUP BY time
        

        CREATE TEMP TABLE double_demeaned AS
        SELECT
            t.unit,
            t.time,
            t.Y,
            t.W - um.mean_W_unit - tm.mean_W_time + om.mean_W AS ddot_W
        FROM panel_data t
        JOIN unit_means um ON t.unit = um.unit
        JOIN time_means tm ON t.time = tm.time
        CROSS JOIN overall_mean om
        

        SELECT
            ddot_W,
            COUNT(*) as count,
            SUM(Y) as sum_Y
        FROM double_demeaned
        GROUP BY ddot_W
        

            SELECT
                ddot_W,
                COUNT(*) as count,
                SUM(Y) a